In [1]:
from utils import *
from encrypt import *

In [2]:
from flask import Flask,request
from flask import make_response
import os
import subprocess
import threading

In [3]:
app = Flask(__name__)
#存储出售数据集的地址，索引是智能合约中，数据集的hash，value是用AES加密后的ipfs地址
AES_KEY = "YIGEMIYAO"
subprocess.getoutput(['touch sell_dict.pkl'])
subprocess.getoutput(['touch prikey_dict.pkl'])

sell_dict = load_model('sell_dict.pkl')
prikey_dict = load_model('prikey_dict.pkl')

account_addr = ""
pwd = ""
@app.route("/api/register")
def register():
    account_addr = request.args.get('account')
    pwd = request.args.get('password')
    return 'success'
@app.route("/account")
def return_account():
    return account_addr
@app.route("/pwd")
def return_pwd():
    return pwd

@app.route("/api/train_and_upload",methods=['GET',])
def train_and_upload():
    #获取到传入url中参数id对应的值
    data_name = request.args.get('data')
    #clf = train_model(data_name)
    model_name = data_name.split('.')[0]+'.pkl'
    #save_model(clf,model_name)
    model_addr = upload_file(model_name)
    data_addr = upload_file(data_name)

    #由于还不知道dataset_hash是什么，暂时先以model_addr作为key，将data_addr的密文存到sell_dict中
    sell_dict[model_addr] = aes_encrypt(data_addr,AES_KEY)
    return model_addr
#使用web3js计算出数据集的hash，更新数据集字典（之前是以model_addr为key，现在将key换为data_hash）
@app.route("/api/update_sell_dict",methods=['GET',])
def update_sell_dict():
    old_key = request.args.get('model_addr')
    new_key = request.args.get('dataset_hash')
    sell_dict[new_key] = sell_dict[old_key]
    sell_dict.pop(old_key)
    return 'success'
@app.route("/api/send_dataset",methods=['GET',])
def send_dataset():
    pub_key,pri_key = rsa.newkeys(256)
    pub_key.n = int(request.args.get('pub_key')[1:])
    dataset_hash = request.args.get('dataset_hash')
    #这里ipfs地址是长度为46的字符串，而python的rsa库只能加密21字节的信息
    #因此需要分别加密
    data_addr = aes_decrypt(sell_dict[dataset_hash],AES_KEY)
    msg1 = data_addr[:6]
    msg2 = data_addr[6:26]
    msg3 = data_addr[26:]
    cipher1 = rsa_encrypt(msg1,pub_key)
    cipher2 = rsa_encrypt(msg2,pub_key)
    cipher3 = rsa_encrypt(msg3,pub_key)
    return '+'.join([cipher1,cipher2,cipher3])
#准备购买数据集，产生用于加密的密钥对，将私钥储存
@app.route("/api/prepare_to_buy")
def prepare_to_buy():
    tx_hash = request.args.get("tx_hash")
    if tx_hash in prikey_dict:
        return 'already request!'
    pub_key,pri_key = rsa.newkeys(256)
    prikey_dict[tx_hash] = pri_key
    return '!' + str(pub_key.n)
#应当一笔交易对应一个密钥
@app.route("/api/update_prikey_dict")
def update_prikey_dict():
    old_key = request.args.get('dataset_hash')
    new_key = request.args.get('tx_hash')
    sell_dict[new_key] = sell_dict[old_key]
    sell_dict.pop(old_key)
    return 'success'

@app.route('/api/decrypt_addr')
def decrypt_addr():
    tx_hash = request.args.get('tx_hash')
    cipher = request.args.get('cipher')
    pri_key = prikey_dict[tx_hash]
    cipher_list = cipher.split('+')
    data_addr = ''
    print(cipher_list)
    for cp in cipher_list:
        print(cp)
        data_addr += rsa_decrypt(cp,pri_key)
    return data_addr

@app.route('/api/train_my_model')
def train_my_model():
    #获取到传入url中参数id对应的值
    data_name = request.args.get('data')
    clf = train_model(train_file)
    model_name = train_file.split('.')[0]+'.pkl'
    save_model(clf,model_name)
    model_addr = upload_file(model_name)
    return model_addr
@app.route('/api/test_dataset')
def test_dataset():
    data_name = request.args.get('data')
    model_addr = request.args.get('model_addr')
    download_file(model_addr)
    clf = load_model(model_addr)
    result = test_model(data_name,clf)
    output = os.popen('rm ' + model_addr)
    return result

@app.route('/')
def hello():
    return 'hello world!'

In [4]:
def run_app():
    app.run('0.0.0.0',7777)

In [5]:
t1 = threading.Thread(target=run_app)
t1.start()

 * Running on http://0.0.0.0:7777/ (Press CTRL+C to quit)


In [11]:
sell_dict

{'0xd43dda362195a4235f82d318b5cf0979bbcd34f2324146a90be8819cd37c5e2b': '2a2dcd9a895a046a3290bbc2826b5f0199ecd1e3692251206a202d20b2bc9efcea7da34a016a09642db0c946ac919376'}

In [12]:
prikey_dict

{'0xd43dda362195a4235f82d318b5cf0979bbcd34f2324146a90be8819cd37c5e2b': PrivateKey(112303242847653819202827252429814041942164067655381248199805039536786058391361, 65537, 94115247980861051193629265685073723558454111880221880152971076657972159552941, 86284406763215830311905694150735646552927, 1301547371772974379227203621895176543)}

127.0.0.1 - - [03/Mar/2020 20:04:50] "GET /api/send_dataset?pub_key=!112303242847653819202827252429814041942164067655381248199805039536786058391361&dataset_hash=0xd43dda362195a4235f82d318b5cf0979bbcd34f2324146a90be8819cd37c5e2b HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2020 20:05:01] "GET /api/test?dataset_hash=0x7671ede9cdaa6666338e5cdd9045516d8fcef0968fe100bd5c45ea3224075137&cipher=391e0366cb6f5739facef3723ac146ac388647fd3aadaa99f3533cd12d374ce4%2B03842344a8944ace6bf207818c267f8145dc35981ed24c6a09e4a73e7ece6cd8%2B95573544c47610dea561570305ddc15d7e38e4946238c716d9812cedad7b355f HTTP/1.1" 404 -
127.0.0.1 - - [03/Mar/2020 20:05:55] "GET /api/register?account=0x86c923e0e2fad5862be4a552be46693ff84aa7cd&password=cmq19950520 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2020 20:05:56] "GET /api/register?account=0x86c923e0e2fad5862be4a552be46693ff84aa7cd&password=cmq19950520 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2020 20:05:57] "GET /api/register?account=0x86c923e0e2fad5862be4a552be46693ff84aa7cd&passwor

In [13]:
@app.route("/api/prepare_to_buy3")
def prepare_to_buy3():
    tx_hash = request.args.get("tx_hash")
    if tx_hash in prikey_dict:
        return 'already request!'
    pub_key,pri_key = rsa.newkeys(256)
    prikey_dict[tx_hash] = pri_key
    return '!' + str(pub_key.n)

127.0.0.1 - - [03/Mar/2020 20:36:40] "GET /api/prepare_to_buy3?tx_hash=0xa089deab18311fb648d2f6cc6c3c630a2d7b9114cd465a76007f5b7a0d08ebae HTTP/1.1" 200 -


In [14]:
prikey_dict

{'0xa089deab18311fb648d2f6cc6c3c630a2d7b9114cd465a76007f5b7a0d08ebae': PrivateKey(88799887105108135218304505068361248532734645768470279283528762347921984664411, 65537, 21264711967706289181928847865219058446597292941550584195021333360767318472561, 67418763349015098392153744611362676740269, 1317139067731140571326869692495719719),
 '0xd43dda362195a4235f82d318b5cf0979bbcd34f2324146a90be8819cd37c5e2b': PrivateKey(112303242847653819202827252429814041942164067655381248199805039536786058391361, 65537, 94115247980861051193629265685073723558454111880221880152971076657972159552941, 86284406763215830311905694150735646552927, 1301547371772974379227203621895176543)}

127.0.0.1 - - [03/Mar/2020 20:55:10] "GET /api/register?account=0x86c923e0e2fad5862be4a552be46693ff84aa7cd&password=cmq19950520 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2020 20:55:34] "GET /api/register?account=0x86c923e0e2fad5862be4a552be46693ff84aa7cd&password=cmq19950520 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2020 20:55:50] "GET /api/register?account=0x86c923e0e2fad5862be4a552be46693ff84aa7cd&password=cmq19950520 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2020 20:55:53] "GET /api/register?account=0x86c923e0e2fad5862be4a552be46693ff84aa7cd&password=cmq19950520 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2020 20:58:27] "GET /api/register?account=0x86c923e0e2fad5862be4a552be46693ff84aa7cd&password=cmq19950520 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2020 20:58:35] "GET /api/register?account=0x86c923e0e2fad5862be4a552be46693ff84aa7cd&password=cmq19950520 HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2020 20:58:47] "GET /api/register?account=0x86c923e0e2fad5862be4a552be46693ff84aa7cd&password=cmq19950520 HTTP/1.1

In [11]:
download_file('QmZiiuXwHfszjy1Spj8UuzGwdQ14ozicEj526ud45agMGA')

In [12]:
import pandas as pd
df = pd.read_csv('QmZiiuXwHfszjy1Spj8UuzGwdQ14ozicEj526ud45agMGA')

In [1]:
from flask import Flask, render_template

app = Flask(__name__,
            static_folder = "./dist/static",
            template_folder = "./dist")

@app.route('/')
def index():
    return render_template("index.html")

In [ ]:
app.run('127.0.0.1',7777)

 * Running on http://127.0.0.1:7777/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Feb/2020 20:59:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2020 20:59:19] "GET /static/css/app.d40fc157.css HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2020 20:59:19] "GET /static/css/chunk-vendors.56b28f4e.css HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2020 20:59:19] "GET /static/js/app.4bb6ab1a.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2020 20:59:19] "GET /static/js/chunk-vendors.99e8beaa.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2020 20:59:20] "GET /static/img/logo.82b9c7a5.png HTTP/1.1" 200 -


In [8]:
@app.route('/api/decrypt_addr')
def decrypt_addr_():
    dataset_hash = request.args.get('dataset_hash')
    cipher = request.args.get('cipher')
    pri_key = prikey_dict[dataset_hash]
    cipher_list = cipher.split(' ')
    data_addr = ''
    print(cipher)
    print(cipher_list)
    for cp in cipher_list:
        print(cp)
        data_addr += rsa_decrypt(cp,pri_key)
    return data_addr

In [10]:
cipher_list = '4ed1fe28e6d54bfb8a80afd50e1a41880ed6bda2dd08a0b4ac713f7b8f35933a+3a68a7a53b8040e5f345280f8dec6627f443d67c9a07698cf16ad1d94cdd7c64+393f4566131a51df0179be4cacc01a60850a0f070624645837e37ad4cb0247c6'.split('+')

In [11]:
cipher_list

['4ed1fe28e6d54bfb8a80afd50e1a41880ed6bda2dd08a0b4ac713f7b8f35933a',
 '3a68a7a53b8040e5f345280f8dec6627f443d67c9a07698cf16ad1d94cdd7c64',
 '393f4566131a51df0179be4cacc01a60850a0f070624645837e37ad4cb0247c6']

In [12]:
rsa_decrypt(cipher_list[0],prikey_dict['chen'])

'QmZiiu'

In [13]:
rsa_decrypt(cipher_list[1],prikey_dict['chen'])

'XwHfszjy1Spj8UuzGwdQ'

In [14]:
rsa_decrypt(cipher_list[2],prikey_dict['chen'])

'14ozicEj526ud45agMGA'

In [157]:
str_test = 'abcdefghijklmnopqrst12'

In [158]:
pub_key,pri_key = rsa.newkeys(256)
rsa_encrypt(str_test,pub_key)

OverflowError: 22 bytes needed for message, but there is only space for 21

In [150]:
model_addr

'QmSXYN1Y9kLrmCSPiSPA3ctcMvnTUxQ8JFZ92zH2diM3BG'

In [151]:
model_addr = 'QmSXYN1Y9kLrmCSPiSPA3ctcMvnTUxQ8JFZ92zH2diM3BG'

In [ ]:
66697993928755715318762783084813522001306220301615457658210060803205040210843

In [14]:
download_file(model_addr)

In [37]:
subprocess.call(['ping','88.88.88.88'])

KeyboardInterrupt: 